In [1]:
import os
import cv2
import math
import types
import random
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report
from torch import optim
import time
from utils import Conv2d
from torch.utils.data import Dataset, DataLoader

In [2]:
# parameters setting
num_classes = 7
cut_size = 225
init_lr = 0.001
batch_size = 64
device= torch.device("cuda")

In [3]:
# load data
data_dir = ''

train_data = torch.load(data_dir + 'pickles/AffectNet_train_full_p1.pt')
train_img, train_y = train_data['images'], train_data['labels']

test_data = torch.load(data_dir + 'pickles/AffectNet_test.pt')
test_img, test_y = test_data['images'], test_data['emotion']


In [4]:
def nine_crops(image, size=[96,96], n_crop=3):
    xx = yy = n_crop
    x = size[0] // xx
    y = size[1] // yy
    crops = ()
    for j in range(yy):
        for i in range(xx):
            left = i*x
            up = y*j
            right = left + x
            low = up + y
            region = image.crop((left,up,right,low))
            region = transforms.Resize([cut_size//n_crop,cut_size//n_crop])(region)
            crops += (region,)       
    return crops

class puzzle_crop(object):
    def __init__(self, size, n_crop):
        self.size = size
        self.n_crop = n_crop

    def __call__(self, img):
        return nine_crops(img, self.size, self.n_crop)


class Lambda(object):
    def __init__(self, lambd):
        assert isinstance(lambd, types.LambdaType)
        self.lambd = lambd
    def __call__(self, img):
        return self.lambd(img)

In [5]:
from torch.utils.data import Dataset, DataLoader
#======= data loaders =======#    
n_crop = 3
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize([int(cut_size*1.1),int(cut_size*1.1)]),
    transforms.RandomCrop([cut_size,cut_size]),
    transforms.RandomHorizontalFlip(),
    puzzle_crop([cut_size,cut_size],3),
    Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),
    puzzle_crop([cut_size,cut_size],3),
    Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
])

resize_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize([cut_size,cut_size]),
    transforms.ToTensor()
])

def default_loader(images):
    img_tensor = train_transform(images)
    return img_tensor

def default_test_loader(images):
    img_tensor = test_transform(images)
    return img_tensor

class Dataset(Dataset):
    def __init__(self,imgs,loader=default_loader):
        self.images = imgs 
        self.loader = loader

    def __getitem__(self, index):

        img = self.images[index]
        n_imgs = tmp = self.loader(img)
        img = resize_transform(img)
        label = torch.randperm(n_crop**2)
        for i in range(label.size(0)):
            n_imgs[i] = tmp[label[i]]
        return img, n_imgs,label

    
    def __len__(self):
        return len(self.images)

In [27]:
class PairPuzzle_AE(nn.Module):

    def __init__(self,model=None,lr=0.0001,num_classes=1000):
        super(PairPuzzle_AE, self).__init__()
        
        self.feature = nn.Sequential(
            # conv_0
            Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            # conv_1
            Conv2d(in_channels=32, out_channels=64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # conv_2
            Conv2d(in_channels=64, out_channels=128, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            # conv_3
            Conv2d(in_channels=128, out_channels=256, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            # conv_4
            Conv2d(in_channels=256, out_channels=512, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(512)
        )
        
        self.fc6 = nn.Sequential()
        self.fc6.add_module('fc6_s1',nn.Linear(12800, 4608))
        self.fc6.add_module('relu6_s1',nn.ReLU(inplace=True))
        self.fc6.add_module('drop6_s1',nn.Dropout(p=0.5))
        self.classifier = nn.Sequential(
#             nn.Linear(9472,4608),
#             nn.ReLU(),
            nn.Linear(4608,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,num_classes)
        )
        
        self.loss_fn = nn.CrossEntropyLoss() #FocalLoss(4.0)
#         self.optimizer=torch.optim.Adam([p for p in self.parameters() if p.requires_grad] , lr=lr)
        self.optimizer=torch.optim.SGD(self.parameters(),lr=lr,momentum=0.9,weight_decay = 5e-4)


    def forward(self,x,n_x):
        B,T,C,H,W = n_x.size()
        x = self.feature(x).view(B,1,-1)
        n_x = n_x.transpose(0,1)
        x_list = []
        for i in range(9):
            z = self.feature(n_x[i])
            z = self.fc6(z.view(B,-1))
            z = z.view([B,1,-1])
            x = torch.cat((z,x),-1)
            x_list.append(z)
        x = torch.cat(x_list,1).view(B*T,-1)
        p = self.classifier(x).view(B,T,-1)
        
        return x,p
        
    def train_model(self, data_loader, num_epochs, lr, min_loss):
        self.train()
        start=time.time()

        for epoch in range(num_epochs):
            running_loss=0
            running_acc = 0.
            total = 0
            num_batches = 0
            for count, (images, n_images, label) in enumerate(data_loader):
                self.optimizer.zero_grad()
                
                images=images.to(device)
                n_images=n_images.to(device)
                label=label.to(device)

                features, scores = self.forward(images, n_images)

                loss =  self.loss_fn(scores, label) 
                loss.backward()

                self.optimizer.step()

                running_loss += loss.detach().item()

                acc = (scores.argmax(-1) == label).float().sum()
                running_acc += acc.item()/9

                total+=images.size(0)
                num_batches += 1
            total_loss = running_loss/num_batches
            if total_loss<min_loss:
                min_loss = total_loss
                print(min_loss)
                torch.save(model.state_dict(),data_dir+'BaselinesGray/CNN_Puzzle_GrayAffectNet_inputsize{}_minLoss.pth'.format(cut_size))
            total_acc = running_acc/total
            elapsed = time.time()-start
            print('\t', 'epoch=',epoch, '\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60),'\t lr=', lr,'\t loss = ', total_loss , '\t Acc = {:.6f}'.format(total_acc*100),'%')
        return min_loss
 

In [ ]:
train_input = Dataset(train_img)
train_loader = DataLoader(train_input, batch_size=16,shuffle=True)

test_input = Dataset(test_img,loader=default_test_loader)
test_loader = DataLoader(test_input, batch_size=batch_size,shuffle=False)

model = PairPuzzle_AE(lr=init_lr,num_classes=n_crop**2)
# model.load_state_dict(torch.load(data_dir + 'BaselinesGray/CNN_PairPuzzle_GrayAffectNet_inputsize225_minLoss.pth'))

model.to(device) 
print(model)
loss = 2.1866

for i in range(1):
    print('Iteration', i)
    loss = model.train_model(data_loader=train_loader, num_epochs=100,lr=init_lr, min_loss=loss)

PairPuzzle_AE(
  (feature): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (7): ReLU()
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): ReLU()
    (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    (13): ReLU()
    (14): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc6): Sequential(
    (fc6_s1): Linear(in_features=12800, out_features=4608, bias=True)
    (relu6_s1): ReLU(inplace)
   

In [ ]:
class PairPuzzle(nn.Module):

    def __init__(self,model=None,lr=0.0001,num_classes=1000):
        super(PairPuzzle, self).__init__()
        
        alexnet = models.alexnet(pretrained=True)
        
        self.feature = alexnet.features
        self.classifier = nn.Sequential(
            nn.Linear(9472,4608),
            nn.ReLU(),
            nn.Linear(4608,4096),
            nn.ReLU(),
            nn.Linear(4096,100),
            nn.ReLU(),
            nn.Linear(100, num_classes)
        )
        
        self.loss_fn = nn.CrossEntropyLoss() #FocalLoss(4.0)
        self.optimizer=torch.optim.Adam([p for p in self.parameters() if p.requires_grad] , lr=lr)
        

    def forward(self,x,n_x):
        bs, n_puzzle, c, h, w = n_x.size() 
        n_x = n_x.view(-1, c, h, w)
        f = self.feature(n_x).view(bs,n_puzzle,-1)
        z = self.feature(x).view(bs,1,-1)
        z = z.expand(bs,n_puzzle,-1)
        f_all = torch.cat((f,z),-1)
        p = self.classifier(f_all)
        return f_all, p.view(bs, n_puzzle, -1)
        
    def train_model(self, data_loader, num_epochs, lr, min_loss):
        self.train()
        start=time.time()

        for epoch in range(num_epochs):
            running_loss=0
            running_acc = 0.
            total = 0
            num_batches = 0
            for count, (images, n_images, label) in enumerate(data_loader):
                self.optimizer.zero_grad()
                
                images=images.to(device)
                n_images=n_images.to(device)
                label=label.to(device)

                features, scores = self.forward(images, n_images)

                loss =  self.loss_fn(scores, label) 
                loss.backward()

                self.optimizer.step()

                running_loss += loss.detach().item()

                acc = (scores.argmax(-1) == label).float().sum()
                running_acc += acc.item()/9

                total+=images.size(0)
                num_batches += 1
            total_loss = running_loss/num_batches
            if total_loss<min_loss:
                min_loss = total_loss
                print(min_loss)
                torch.save(model.state_dict(),data_dir+'BaselinesGray/Puzzle_GrayAffectNet_inputsize{}_minLoss.pth'.format(cut_size))
            total_acc = running_acc/total
            elapsed = time.time()-start
            print('\t', 'epoch=',epoch, '\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60),'\t lr=', lr,'\t loss = ', total_loss , '\t Acc = {:.6f}'.format(total_acc*100),'%')
        return min_loss

In [ ]:
train_input = Dataset(train_img)
train_loader = DataLoader(train_input, batch_size=batch_size,shuffle=True)

test_input = Dataset(test_img,loader=default_test_loader)
test_loader = DataLoader(test_input, batch_size=batch_size,shuffle=False)

model = PairPuzzle(lr=init_lr,num_classes=n_crop**2)
model.to(device)
print(model)
loss = 5.

for i in range(1):
    print('Iteration', i)
    loss = model.train_model(data_loader=train_loader, num_epochs=100,lr=init_lr, min_loss=loss)